<a href="https://colab.research.google.com/github/takuya-tokumoto/polar_tutorial/blob/main/polars%E3%83%81%E3%83%A5%E3%83%BC%E3%83%88%E3%83%AA%E3%82%A2%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

本稿は主に以下を参照してます。
- 前処理大全［データ分析のためのSQL/R/Python実践テクニック］本橋 智光 (著)
- [polars公式ガイド](https://pola-rs.github.io/polars-book/user-guide/index.html)
- [Qiita記事 pandasから移行する人向け polars使用ガイド](https://qiita.com/nkay/items/9cfb2776156dc7e054c8#%E8%A1%8C%E3%83%95%E3%82%A3%E3%83%AB%E3%82%BF%E3%83%AA%E3%83%B3%E3%82%B0filter)
- [Qiita記事 超高速…だけじゃない！Pandasに代えてPolarsを使いたい理由](https://qiita.com/_jinta/items/fac13f09e8e8a5769b79#)

# polarとは？

- Polarsは、Apache Arrow Columnar Formatをメモリモデルとして、Rustで実装された超高速DataFramesライブラリです。

- Blazingly fast
    - Polarsは非常に高速です。実際、利用可能なソリューションの中で最高のパフォーマンスを発揮します。h2oaiのdb-benchmarkで結果をご覧ください。
    - TPCHベンチマークでは、polarsはpandas、dask、modin、vaexよりも（IOを含む）フルクエリで数桁速いです。
- Lightweight
    - Polarsはまた、非常に軽量です。必要な依存関係はゼロで、これはインポート時間にも表れています。
            ○ Polars: 70ms
            ○ numpy: 104ms
            ○ pandas：520ms
- Handles larger than RAM data
    - メモリに収まらないデータがある場合、polars lazyはストリーミングでクエリ（またはクエリの一部）を処理することができ、必要なメモリを大幅に削減できるため、250GBのデータセットをラップトップで処理できるかもしれません。
    - collect(streaming=True) を使ってクエリをストリーミングで実行します。(これは少し遅くなるかもしれませんが、それでも非常に速いです!)


#所感

*   pandasよりもpoloarsの方が基本優秀
*   エクスプレションを使うことでさらに処理速度が上がる
*   書き方はpandasよりもpysparkに近い印象

*   pandasにできることはほぼできるが痒いところに手が届いていない印象
    *   csvへの書き出しの際にutf-8以外選択できない
    *   `.hist()`などpandasから気軽に使えた描画ツール系は対応していないっぽい
    *   pandasにあったindexが存在しないため`.loc`, `.iloc`などの機能が廃止(逆にpandasのindex機能がうざかったので自分的には使いやすい。。)
    *   欠損値の取り扱いについて、pandasではNA(pd.NA)、polarsではnull(pl.Null)が割り当てられます。
        *   pandasでNAが入ったカラムはobject型になりますが、polarsではnull以外の型がキープされます。例えば、`pl.Series=[1, null, 2]`であればi64型になります。
        *   nullまたはNaNに対する数値演算・比較演算ではnullまたはNaNが維持されます。



# pl.Exprについて
Polarsの書きやすさの中心にあるのが、**polars.Expression**というやつです。これは何かというと、

> a mapping from a series to a series

と説明されてます[^expl]。つまり、**ある列から他の列への加工処理の方法**を記述したものです。複数列からのmappingもできます。データフレームに対する主な処理は `select`, `with_columns`, `agg`, `filter` などですが、**任意の場所でpl.Exprを使うことが出来ます**。どういうことか。

[^expl]: https://pola-rs.github.io/polars-book/user-guide/dsl/intro.html

たとえば、文字列で入ってる `cost` 列を整数に変換する処理は
```python
pl.col("cost").str.extract("\$(.*)").cast(pl.Int64)
```
という pl.Expr で書けます（まず `pl.col` でcost列を指定し、それに対し `str.extract` で `$` に続く部分を抽出する文字列処理を行い、最後に `cast` でInt型に変換しています）。これを色んな所で使いまわせます。

```python
# 整数に変換した列の追加
df.with_columns(pl.col("cost").str.extract("\$(.*)").cast(pl.Int64).alias("cost_int"))

# コストが100ドル以上の行を選択
df.filter(pl.col("cost").str.extract("\$(.*)").cast(pl.Int64) > 100)

# 店舗ごとのコストの合計を計算
df.groupby("store").agg(pl.col("cost").str.extract("\$(.*)").cast(pl.Int64).sum())
```

ここで強調したいのは、同じ表現を繰り返し使えることではありません。ポイントは、**どんな処理も同じ頭の働かせ方で済む**ことです。書き方に悩むことがなくなり、統一感も出しやすいです。

このようにpl.Expressionを用いると柔軟な記述が実現できるため、公式でもExpression APIを推奨しています。

# データ型について



`polars`のシリーズは、`pandas`のそれと同様、データ型（`dtype`）が存在します。以下はデータ型の例です。

| polarsの型                  | pandas(Numpy)の類似する型      | 説明             |
| :-------------------------- | :----------------------------- | :--------------- |
| `pl.Int8` ~ `pl.Int64`      | `np.int8` ~ `np.int64` 等      | 整数             |
| `pl.UInt8` ~ `pl.UInt64`    | `np.uint8` ~ `np.uint64` 等    | 整数（符号なし） |
| `pl.Float32` / `pl.Float64` | `np.float32` / `np.float64` 等 | 浮動小数         |
| `pl.Boolean`                | `np.bool` / `pd.BooleanDtype`  | 真偽値           |
| `pl.Utf8`                   | `pd.StringDtype`               | 文字列           |
| `pl.Categorical`            | `pd.CategoricalDtype`          | カテゴリー       |
| `pl.List`                   | -                              | リスト           |
| `pl.Struct`                 | -                              | 構造化配列       |
| `pl.Date`                   | -                              | 日付             |
| `pl.Time`                   | -                              | 時刻（日付無し） |
| `pl.Datetime`               | `np.datetime64`                | 日時             |
| `pl.Duration`               | `np.timedelta64`               | 時間（時刻の差） |
| `pl.Object`                 | `np.object_`                   | オブジェクト     |

- pandasではデフォルトでは文字列をオブジェクトデータ型で扱うようになっています。それに対してpolarsには最初から文字列専用のデータ型が用意されています。
- pandasで日付を扱う場合は一般的には時刻を0:00:00にした日時`np.datetime64`で代用しますが、polarsには日付のみを扱う`pl.Date`が存在し、また時刻のみを扱う`pl.Time`、日付＋時刻を扱う`pl.Datetime`もそれぞれ存在します。
- 型変換メソッド、つまりpandasの`.astype()`は、polarsでは`.cast()`です。

# Import 

- condaパッケージ(conda install polars)もありますが、Polarsのインストールはpipが好ましい。
- 依存関係を考慮した上でPolarsがインストールされる。

In [1]:
pip install -U polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 53.5 MB/s eta 0:00:00


In [2]:
import polars as pl
import pandas as pd

# to enrich the examples in this quickstart with dates
from datetime import datetime, timedelta 
# to generate data for the examples
import numpy as np 

print('polars version: ',pl.__version__)

polars version:  0.16.2


# データフレーム作成

　- polarとpandasで書き方変化なし

## polars

In [3]:
# rng = np.random.default_rng(0)
pldf_a = pl.DataFrame({"integer": [1, 2, 3], 
                          "date": [
                              (datetime(2022, 1, 1)), 
                              (datetime(2022, 1, 2)), 
                              (datetime(2022, 1, 3))
                          ], 
                          "float":[4.0, 5.0, 6.0]})

pldf_b = pl.Series("a", [1, 2, 3, 4, 5])

In [4]:
pldf_a

integer,date,float
i64,datetime[μs],f64
1,2022-01-01 00:00:00,4.0
2,2022-01-02 00:00:00,5.0
3,2022-01-03 00:00:00,6.0


In [5]:
pldf_b

a
i64
1
2
3
4
5


## pandas

In [6]:
pdf_a = pd.DataFrame({"integer": [1, 2, 3], 
                          "date": [
                              (datetime(2022, 1, 1)), 
                              (datetime(2022, 1, 2)), 
                              (datetime(2022, 1, 3))
                          ], 
                          "float":[4.0, 5.0, 6.0]})

pdf_b = pd.Series("a",[1, 2, 3, 4, 5])

In [7]:
pdf_a

,integer,date,float
0,1,2022-01-01,4.0
1,2,2022-01-02,5.0
2,3,2022-01-03,6.0


In [8]:
pdf_b

1    a
2    a
3    a
4    a
5    a
dtype: object

# ファイルの読み書き

- https://github.com/ghmagazine/awesomebook/tree/master/data 配下のファイルをノートブックと同じ階層へアップロードする

In [9]:
!mkdir tmp

## polars

- polarsのCSVファイル出力関数はエンコードを指定できない（UTF8が強制される）。

In [10]:
%%time

pl_reserve_tb = pl.read_csv('reserve.csv', encoding='UTF-8')
pl_hotel_tb = pl.read_csv('hotel.csv', encoding='UTF-8')
pl_customer_tb = pl.read_csv('customer.csv', encoding='UTF-8')

pl_reserve_tb.write_csv('tmp/tmp_reserve.csv')

CPU times: user 82.7 ms, sys: 8.82 ms, total: 91.5 ms
Wall time: 91 ms


## pandas

In [11]:
%%time

pd_reserve_tb = pd.read_csv('reserve.csv', encoding='UTF-8')
pd_hotel_tb = pd.read_csv('hotel.csv', encoding='UTF-8') 
pd_customer_tb = pd.read_csv('customer.csv', encoding='UTF-8')

pd_reserve_tb.to_csv('tmp/tmp_reserve.csv')

CPU times: user 26.9 ms, sys: 9.02 ms, total: 36 ms
Wall time: 40.7 ms


# 可視化

## polars

.head() .tail()

In [12]:
pl_reserve_tb.head()

reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
str,str,str,str,str,str,str,i64,i64
"""r1""","""h_75""","""c_1""","""2016-03-06 13:...","""2016-03-26""","""10:00:00""","""2016-03-29""",4,97200
"""r2""","""h_219""","""c_1""","""2016-07-16 23:...","""2016-07-20""","""11:30:00""","""2016-07-21""",2,20600
"""r3""","""h_179""","""c_1""","""2016-09-24 10:...","""2016-10-19""","""09:00:00""","""2016-10-22""",2,33600
"""r4""","""h_214""","""c_1""","""2017-03-08 03:...","""2017-03-29""","""11:00:00""","""2017-03-30""",4,194400
"""r5""","""h_16""","""c_1""","""2017-09-05 19:...","""2017-09-22""","""10:30:00""","""2017-09-23""",3,68100


In [13]:
pl_reserve_tb.tail()

reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
str,str,str,str,str,str,str,i64,i64
"""r4026""","""h_129""","""c_999""","""2017-06-27 23:...","""2017-07-10""","""09:30:00""","""2017-07-11""",2,16000
"""r4027""","""h_97""","""c_999""","""2017-09-29 05:...","""2017-10-09""","""10:30:00""","""2017-10-10""",2,41800
"""r4028""","""h_27""","""c_999""","""2018-03-14 05:...","""2018-04-02""","""11:30:00""","""2018-04-04""",2,74800
"""r4029""","""h_48""","""c_1000""","""2016-04-16 15:...","""2016-05-10""","""09:30:00""","""2016-05-13""",4,540000
"""r4030""","""h_117""","""c_1000""","""2016-06-06 08:...","""2016-07-06""","""09:00:00""","""2016-07-09""",1,44100


.glimpse()  


*   文字列として、各列の先頭の要素10個が表示されます。
*   列数が多い場合、print()よりもこちらのほうが見やすいとされています。



In [14]:
print(pl_reserve_tb.glimpse())

Rows: 4030
Columns: 9
$ reserve_id       <str> r1, r2, r3, r4, r5, r6, r7, r8, r9, r10
$ hotel_id         <str> h_75, h_219, h_179, h_214, h_16, h_241, h_256, h_241, h_217, h_240
$ customer_id      <str> c_1, c_1, c_1, c_1, c_1, c_1, c_1, c_1, c_2, c_2
$ reserve_datetime <str> 2016-03-06 13:09:42, 2016-07-16 23:39:55, 2016-09-24 10:03:17, 2017-03-08 03:20:10, 2017-09-05 19:50:37, 2017-11-27 18:47:05, 2017-12-29 10:38:36, 2018-05-26 08:42:51, 2016-03-05 13:31:06, 2016-06-25 09:12:22
$ checkin_date     <str> 2016-03-26, 2016-07-20, 2016-10-19, 2017-03-29, 2017-09-22, 2017-12-04, 2018-01-25, 2018-06-08, 2016-03-25, 2016-07-14
$ checkin_time     <str> 10:00:00, 11:30:00, 09:00:00, 11:00:00, 10:30:00, 12:00:00, 10:30:00, 10:00:00, 09:30:00, 11:00:00
$ checkout_date    <str> 2016-03-29, 2016-07-21, 2016-10-22, 2017-03-30, 2017-09-23, 2017-12-06, 2018-01-28, 2018-06-09, 2016-03-27, 2016-07-17
$ people_num       <i64> 4, 2, 2, 4, 3, 3, 1, 1, 3, 4
$ total_price      <i64> 97200, 20600, 33600, 1

.describe()


*   polarではint,float列以外もdescribeの結果が表示される


In [15]:
pl_reserve_tb.describe()

describe,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
str,str,str,str,str,str,str,str,f64,f64
"""count""","""4030""","""4030""","""4030""","""4030""","""4030""","""4030""","""4030""",4030.0,4030.0
"""null_count""","""0""","""0""","""0""","""0""","""0""","""0""","""0""",0.0,0.0
"""mean""",null,null,null,null,null,null,null,2.542184,103065.955335
"""std""",null,null,null,null,null,null,null,1.120925,110288.484355
"""min""","""r1""","""h_1""","""c_1""","""2016-01-01 08:...","""2016-01-02""","""09:00:00""","""2016-01-04""",1.0,3500.0
"""max""","""r999""","""h_99""","""c_999""","""2019-02-25 03:...","""2019-03-13""","""12:30:00""","""2019-03-15""",4.0,897600.0
"""median""",null,null,null,null,null,null,null,3.0,64800.0


## pandas

.head() .tail()

In [16]:
pd_reserve_tb.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


In [17]:
pd_reserve_tb.tail()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
4025,r4026,h_129,c_999,2017-06-27 23:00:02,2017-07-10,09:30:00,2017-07-11,2,16000
4026,r4027,h_97,c_999,2017-09-29 05:24:57,2017-10-09,10:30:00,2017-10-10,2,41800
4027,r4028,h_27,c_999,2018-03-14 05:01:45,2018-04-02,11:30:00,2018-04-04,2,74800
4028,r4029,h_48,c_1000,2016-04-16 15:20:17,2016-05-10,09:30:00,2016-05-13,4,540000
4029,r4030,h_117,c_1000,2016-06-06 08:16:51,2016-07-06,09:00:00,2016-07-09,1,44100


.glimpse()  


*   対応するpandas methodはなさそう



.describe()

In [18]:
pd_reserve_tb.describe()

,people_num,total_price
count,4030.000000,4030.000000
mean,2.542184,103065.955335
std,1.120925,110288.484355
min,1.000000,3500.000000
25%,2.000000,32400.000000
50%,3.000000,64800.000000
75%,4.000000,129600.000000
max,4.000000,897600.000000


# 条件指定による抽出

## polars

`[]`を使った行の選択

*   行選択はdf[]の[]に整数を渡すことで行います（Python標準のリストと同様）。
*   polarsにはインデックス列が存在しないため、pandasと違い.locや.ilocは存在しません。


In [19]:
pl_reserve_tb[1:3,:]

# pd_reserve_tb.iloc[1:3,:]

reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
str,str,str,str,str,str,str,i64,i64
"""r2""","""h_219""","""c_1""","""2016-07-16 23:...","""2016-07-20""","""11:30:00""","""2016-07-21""",2,20600
"""r3""","""h_179""","""c_1""","""2016-09-24 10:...","""2016-10-19""","""09:00:00""","""2016-10-22""",2,33600


.filter()



*   条件にもとづいて特定の行を抽出する場合は.filter()メソッドを用います。
*   pandasの`df[df["column"] > 3]`のような書き方はできません。
*   .filter()メソッドでは通常、エクスプレッション`（pl.Expr()）`を渡すことでフィルタリングします。



In [20]:
pl_reserve_tb.filter((pl.col("checkout_date") >= '2016-10-13') & (pl.col("checkout_date") <= '2016-10-14'))

reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
str,str,str,str,str,str,str,i64,i64
"""r285""","""h_121""","""c_67""","""2016-09-27 06:...","""2016-10-12""","""12:00:00""","""2016-10-14""",4,184000
"""r514""","""h_74""","""c_120""","""2016-10-06 03:...","""2016-10-11""","""12:30:00""","""2016-10-14""",2,28800
"""r1066""","""h_205""","""c_261""","""2016-09-14 02:...","""2016-10-11""","""10:00:00""","""2016-10-14""",4,85200
"""r1481""","""h_116""","""c_364""","""2016-09-17 17:...","""2016-10-11""","""11:30:00""","""2016-10-13""",4,107200
"""r1547""","""h_149""","""c_377""","""2016-09-27 08:...","""2016-10-10""","""11:00:00""","""2016-10-13""",2,153600
"""r1710""","""h_59""","""c_422""","""2016-09-19 04:...","""2016-10-10""","""12:00:00""","""2016-10-13""",2,148800
"""r1933""","""h_113""","""c_477""","""2016-09-24 09:...","""2016-10-12""","""11:30:00""","""2016-10-13""",4,77200
"""r2059""","""h_9""","""c_517""","""2016-09-19 15:...","""2016-10-11""","""12:30:00""","""2016-10-13""",3,188400
"""r2116""","""h_77""","""c_527""","""2016-10-05 00:...","""2016-10-11""","""09:00:00""","""2016-10-13""",4,353600


.is_in()：リストに含まれるかどうかを判定



*   なおpandasの`.isin`と違い、polarsの`.is_in()`メソッドにはセットやndarrayを渡すことはできないようです。



In [21]:
pl_reserve_tb.filter(pl.col('people_num').is_in([3,4]))

reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
str,str,str,str,str,str,str,i64,i64
"""r1""","""h_75""","""c_1""","""2016-03-06 13:...","""2016-03-26""","""10:00:00""","""2016-03-29""",4,97200
"""r4""","""h_214""","""c_1""","""2017-03-08 03:...","""2017-03-29""","""11:00:00""","""2017-03-30""",4,194400
"""r5""","""h_16""","""c_1""","""2017-09-05 19:...","""2017-09-22""","""10:30:00""","""2017-09-23""",3,68100
"""r6""","""h_241""","""c_1""","""2017-11-27 18:...","""2017-12-04""","""12:00:00""","""2017-12-06""",3,36000
"""r9""","""h_217""","""c_2""","""2016-03-05 13:...","""2016-03-25""","""09:30:00""","""2016-03-27""",3,68400
"""r10""","""h_240""","""c_2""","""2016-06-25 09:...","""2016-07-14""","""11:00:00""","""2016-07-17""",4,320400
"""r12""","""h_268""","""c_2""","""2017-05-24 10:...","""2017-06-20""","""09:00:00""","""2017-06-21""",4,81600
"""r16""","""h_135""","""c_2""","""2018-07-06 04:...","""2018-07-08""","""10:00:00""","""2018-07-09""",4,46400
"""r19""","""h_23""","""c_3""","""2017-01-11 22:...","""2017-02-08""","""10:00:00""","""2017-02-10""",3,390600


## pandas

`[]`を使った行の選択

In [22]:
# pd_reserve_tb[1:3,:]

pd_reserve_tb.iloc[1:3,:]

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600


条件文による抽出

In [23]:
pd_reserve_tb[(pd_reserve_tb['checkout_date'] >= '2016-10-13') &
              (pd_reserve_tb['checkout_date'] <= '2016-10-14')]

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
284,r285,h_121,c_67,2016-09-27 06:13:19,2016-10-12,12:00:00,2016-10-14,4,184000
513,r514,h_74,c_120,2016-10-06 03:12:04,2016-10-11,12:30:00,2016-10-14,2,28800
1065,r1066,h_205,c_261,2016-09-14 02:57:59,2016-10-11,10:00:00,2016-10-14,4,85200
1480,r1481,h_116,c_364,2016-09-17 17:45:39,2016-10-11,11:30:00,2016-10-13,4,107200
1546,r1547,h_149,c_377,2016-09-27 08:19:24,2016-10-10,11:00:00,2016-10-13,2,153600
1709,r1710,h_59,c_422,2016-09-19 04:17:25,2016-10-10,12:00:00,2016-10-13,2,148800
1932,r1933,h_113,c_477,2016-09-24 09:04:26,2016-10-12,11:30:00,2016-10-13,4,77200
2058,r2059,h_9,c_517,2016-09-19 15:32:35,2016-10-11,12:30:00,2016-10-13,3,188400
2115,r2116,h_77,c_527,2016-10-05 00:44:09,2016-10-11,09:00:00,2016-10-13,4,353600
2170,r2171,h_177,c_540,2016-09-28 01:21:26,2016-10-11,10:00:00,2016-10-13,4,560800


.isin()：リストに含まれるかどうかを判定

In [24]:
pd_reserve_tb[pd_reserve_tb['people_num'].isin([3,4])]

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100
5,r6,h_241,c_1,2017-11-27 18:47:05,2017-12-04,12:00:00,2017-12-06,3,36000
8,r9,h_217,c_2,2016-03-05 13:31:06,2016-03-25,09:30:00,2016-03-27,3,68400
...,...,...,...,...,...,...,...,...,...
4018,r4019,h_224,c_996,2017-12-10 19:39:53,2017-12-20,11:30:00,2017-12-23,3,36000
4021,r4022,h_172,c_997,2016-05-13 09:08:55,2016-06-06,09:30:00,2016-06-08,3,96000
4022,r4023,h_4,c_999,2016-04-21 21:50:14,2016-04-22,12:00:00,2016-04-25,4,62400
4023,r4024,h_244,c_999,2016-10-06 18:01:34,2016-10-07,11:30:00,2016-10-09,3,59400


# ユニーク行・重複行

## polars

*    unique()は、pandasの.drop_duplicates()に相当し、重複データを削除することができます。

*    subset=引数：識別する列を指定します。デフォルトでは、全ての列の値が一致している行のみが削除されます。
*    keep=引数：デフォルトのkeep="first"では、重複行のうち最初に出現する行は残されます。keep="last"を指定すると下の方にある行が残ります。
*    keep="none"を指定すると重複行をすべて削除します。

In [25]:
def dup_counts_pl(df, target_col):
    # tmp = df[target_col].is_duplicated().value_counts() #.shape[0] True以外もカウントしてしまうため×
    tmp = df.select([(pl.col(target_col).count() - pl.col(target_col).n_unique())]).get_columns()[0][0]

    print(f'{target_col} 重複件数: ', tmp)

In [26]:
dup_counts_pl(pl_reserve_tb, 'hotel_id')

hotel_id_uu = pl_reserve_tb.unique(subset=['hotel_id'])

dup_counts_pl(hotel_id_uu, 'hotel_id')

hotel_id 重複件数:  3730
hotel_id 重複件数:  0


## pandas

In [27]:
def dup_counts_pd(pdf, target_col):
    tmp = pdf[target_col][pdf[target_col].duplicated()].count()

    print(f'{target_col} 重複件数: ', tmp)

In [28]:
dup_counts_pd(pd_reserve_tb, 'hotel_id')

hotel_id_uu = pd_reserve_tb.drop_duplicates(subset=['hotel_id'])

dup_counts_pd(hotel_id_uu, 'hotel_id')

hotel_id 重複件数:  3730
hotel_id 重複件数:  0


# 列の選択

## polars



*   データフレーム内の特定のシリーズ（1列）を選択する場合は、.get_column()メソッドに列名を渡します。



In [29]:
# データフレーム内の特定のシリーズ（1列）を選択する場合は、.get_column()メソッドに列名を渡します。
pl_reserve_tb.get_column('hotel_id')

# .get_columns()を使うことでdartaframeの列を分解し、シリーズのリストに変換する
pl_reserve_tb.get_columns()

[shape: (4030,)
 Series: 'reserve_id' [str]
 [
 	"r1"
 	"r2"
 	"r3"
 	"r4"
 	"r5"
 	"r6"
 	"r7"
 	"r8"
 	"r9"
 	"r10"
 	"r11"
 	"r12"
 	...
 	"r4018"
 	"r4019"
 	"r4020"
 	"r4021"
 	"r4022"
 	"r4023"
 	"r4024"
 	"r4025"
 	"r4026"
 	"r4027"
 	"r4028"
 	"r4029"
 	"r4030"
 ], shape: (4030,)
 Series: 'hotel_id' [str]
 [
 	"h_75"
 	"h_219"
 	"h_179"
 	"h_214"
 	"h_16"
 	"h_241"
 	"h_256"
 	"h_241"
 	"h_217"
 	"h_240"
 	"h_183"
 	"h_268"
 	...
 	"h_204"
 	"h_224"
 	"h_243"
 	"h_159"
 	"h_172"
 	"h_4"
 	"h_244"
 	"h_160"
 	"h_129"
 	"h_97"
 	"h_27"
 	"h_48"
 	"h_117"
 ], shape: (4030,)
 Series: 'customer_id' [str]
 [
 	"c_1"
 	"c_1"
 	"c_1"
 	"c_1"
 	"c_1"
 	"c_1"
 	"c_1"
 	"c_1"
 	"c_2"
 	"c_2"
 	"c_2"
 	"c_2"
 	...
 	"c_996"
 	"c_996"
 	"c_996"
 	"c_997"
 	"c_997"
 	"c_999"
 	"c_999"
 	"c_999"
 	"c_999"
 	"c_999"
 	"c_999"
 	"c_1000"
 	"c_1000"
 ], shape: (4030,)
 Series: 'reserve_datetime' [str]
 [
 	"2016-03-06 13:...
 	"2016-07-16 23:...
 	"2016-09-24 10:...
 	"2017-03-08 03:...
 	"2017-

In [30]:
# .get_columns()を使うことでdartaframeの列を分解し、シリーズのリストに変換する
pl_reserve_tb.get_columns()

pl_reserve_tb.get_columns()[1]

hotel_id
str
"""h_75"""
"""h_219"""
"""h_179"""
"""h_214"""
"""h_16"""
"""h_241"""
"""h_256"""
"""h_241"""
"""h_217"""


In [31]:
pl_reserve_tb.columns

['reserve_id',
 'hotel_id',
 'customer_id',
 'reserve_datetime',
 'checkin_date',
 'checkin_time',
 'checkout_date',
 'people_num',
 'total_price']

In [32]:
# .selectを使った場合はget_columnsと異なりdataframeを返す
# 記法の互換性のため、pandasのdf["column"]のような操作も実装されていますが、推奨されていません。

pl_reserve_tb.select(['reserve_id','hotel_id','customer_id'])

reserve_id,hotel_id,customer_id
str,str,str
"""r1""","""h_75""","""c_1"""
"""r2""","""h_219""","""c_1"""
"""r3""","""h_179""","""c_1"""
"""r4""","""h_214""","""c_1"""
"""r5""","""h_16""","""c_1"""
"""r6""","""h_241""","""c_1"""
"""r7""","""h_256""","""c_1"""
"""r8""","""h_241""","""c_1"""
"""r9""","""h_217""","""c_2"""


## pandas

In [33]:
# selectはない
pd_reserve_tb[['reserve_id','hotel_id','customer_id']]

,reserve_id,hotel_id,customer_id
0,r1,h_75,c_1
1,r2,h_219,c_1
2,r3,h_179,c_1
3,r4,h_214,c_1
4,r5,h_16,c_1
...,...,...,...
4025,r4026,h_129,c_999
4026,r4027,h_97,c_999
4027,r4028,h_27,c_999
4028,r4029,h_48,c_1000


# 列の追加

## polars

In [34]:
pl_reserve_tb.head()

reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
str,str,str,str,str,str,str,i64,i64
"""r1""","""h_75""","""c_1""","""2016-03-06 13:...","""2016-03-26""","""10:00:00""","""2016-03-29""",4,97200
"""r2""","""h_219""","""c_1""","""2016-07-16 23:...","""2016-07-20""","""11:30:00""","""2016-07-21""",2,20600
"""r3""","""h_179""","""c_1""","""2016-09-24 10:...","""2016-10-19""","""09:00:00""","""2016-10-22""",2,33600
"""r4""","""h_214""","""c_1""","""2017-03-08 03:...","""2017-03-29""","""11:00:00""","""2017-03-30""",4,194400
"""r5""","""h_16""","""c_1""","""2017-09-05 19:...","""2017-09-22""","""10:30:00""","""2017-09-23""",3,68100


In [35]:
tmp = pl_reserve_tb

# 新しいシリーズの作成を伴う操作
new_seires = (tmp.get_column("total_price") * 2).alias("total_price_2x")
tmp.with_columns(new_seires)

# エクスプレッションによる操作(こちらの方がおすすめ)
new_column_2x = (pl.col("total_price") * 2).alias("total_price_2x")
tmp.with_columns(new_column_2x)

# .with_columns()に列のリストを渡すと、一度に複数の列を追加
new_column_3x = (pl.col("total_price") * 3).alias("total_price_3x")
tmp.with_columns([new_column_2x, new_column_3x])

reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,total_price_2x,total_price_3x
str,str,str,str,str,str,str,i64,i64,i64,i64
"""r1""","""h_75""","""c_1""","""2016-03-06 13:...","""2016-03-26""","""10:00:00""","""2016-03-29""",4,97200,194400,291600
"""r2""","""h_219""","""c_1""","""2016-07-16 23:...","""2016-07-20""","""11:30:00""","""2016-07-21""",2,20600,41200,61800
"""r3""","""h_179""","""c_1""","""2016-09-24 10:...","""2016-10-19""","""09:00:00""","""2016-10-22""",2,33600,67200,100800
"""r4""","""h_214""","""c_1""","""2017-03-08 03:...","""2017-03-29""","""11:00:00""","""2017-03-30""",4,194400,388800,583200
"""r5""","""h_16""","""c_1""","""2017-09-05 19:...","""2017-09-22""","""10:30:00""","""2017-09-23""",3,68100,136200,204300
"""r6""","""h_241""","""c_1""","""2017-11-27 18:...","""2017-12-04""","""12:00:00""","""2017-12-06""",3,36000,72000,108000
"""r7""","""h_256""","""c_1""","""2017-12-29 10:...","""2018-01-25""","""10:30:00""","""2018-01-28""",1,103500,207000,310500
"""r8""","""h_241""","""c_1""","""2018-05-26 08:...","""2018-06-08""","""10:00:00""","""2018-06-09""",1,6000,12000,18000
"""r9""","""h_217""","""c_2""","""2016-03-05 13:...","""2016-03-25""","""09:30:00""","""2016-03-27""",3,68400,136800,205200


## pandas

In [36]:
tmp = pd_reserve_tb

tmp['total_price_2x'] = tmp['total_price']*2
tmp['total_price_3x'] = tmp['total_price']*3
tmp

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,total_price_2x,total_price_3x
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,194400,291600
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,41200,61800
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,67200,100800
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,388800,583200
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,136200,204300
...,...,...,...,...,...,...,...,...,...,...,...
4025,r4026,h_129,c_999,2017-06-27 23:00:02,2017-07-10,09:30:00,2017-07-11,2,16000,32000,48000
4026,r4027,h_97,c_999,2017-09-29 05:24:57,2017-10-09,10:30:00,2017-10-10,2,41800,83600,125400
4027,r4028,h_27,c_999,2018-03-14 05:01:45,2018-04-02,11:30:00,2018-04-04,2,74800,149600,224400
4028,r4029,h_48,c_1000,2016-04-16 15:20:17,2016-05-10,09:30:00,2016-05-13,4,540000,1080000,1620000


# 結合

## polars

In [37]:
tmp_left = pl_reserve_tb.filter(pl.col('people_num')==1)
tmp_right =  pl_hotel_tb.filter(pl.col('is_business')==True)

tmp_left.join(tmp_right, on = 'hotel_id', how='inner')

reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business
str,str,str,str,str,str,str,i64,i64,i64,str,str,f64,f64,bool
"""r7""","""h_256""","""c_1""","""2017-12-29 10:...","""2018-01-25""","""10:30:00""","""2018-01-28""",1,103500,34500,"""C""","""C-1""",38.237294,140.696131,true
"""r11""","""h_183""","""c_2""","""2016-11-19 12:...","""2016-12-08""","""11:00:00""","""2016-12-11""",1,29700,9900,"""G""","""G-4""",33.595248,130.633567,true
"""r13""","""h_223""","""c_2""","""2017-10-19 03:...","""2017-10-21""","""09:30:00""","""2017-10-23""",1,137000,68500,"""C""","""C-2""",38.329097,140.698165,true
"""r18""","""h_132""","""c_3""","""2016-10-22 02:...","""2016-11-12""","""12:00:00""","""2016-11-13""",1,20400,20400,"""C""","""C-1""",38.231842,140.797268,true
"""r25""","""h_277""","""c_4""","""2016-03-28 07:...","""2016-04-07""","""10:30:00""","""2016-04-10""",1,39300,13100,"""C""","""C-1""",38.233985,140.795603,true
"""r26""","""h_132""","""c_4""","""2016-05-11 17:...","""2016-06-05""","""11:30:00""","""2016-06-06""",1,20400,20400,"""C""","""C-1""",38.231842,140.797268,true
"""r32""","""h_287""","""c_4""","""2017-11-02 19:...","""2017-11-05""","""10:00:00""","""2017-11-07""",1,29000,14500,"""G""","""G-4""",33.596284,130.638578,true
"""r34""","""h_273""","""c_5""","""2016-11-25 15:...","""2016-12-19""","""12:00:00""","""2016-12-22""",1,134700,44900,"""A""","""A-3""",35.809547,139.940654,true
"""r35""","""h_90""","""c_5""","""2017-03-30 13:...","""2017-04-05""","""11:30:00""","""2017-04-07""",1,16000,8000,"""A""","""A-3""",35.915943,139.931772,true


## pandas

In [38]:
pd.merge(pd_reserve_tb.query('people_num == 1'),
         pd_hotel_tb.query('is_business'),
         on = 'hotel_id', how = 'inner')

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,total_price_2x,total_price_3x,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business
0,r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,207000,310500,34500,C,C-1,38.237294,140.696131,True
1,r997,h_256,c_244,2016-10-15 22:47:40,2016-10-31,10:30:00,2016-11-02,1,69000,138000,207000,34500,C,C-1,38.237294,140.696131,True
2,r2602,h_256,c_650,2016-05-10 00:42:56,2016-05-12,11:00:00,2016-05-14,1,69000,138000,207000,34500,C,C-1,38.237294,140.696131,True
3,r3738,h_256,c_930,2017-04-12 09:53:00,2017-05-08,11:30:00,2017-05-09,1,34500,69000,103500,34500,C,C-1,38.237294,140.696131,True
4,r11,h_183,c_2,2016-11-19 12:49:10,2016-12-08,11:00:00,2016-12-11,1,29700,59400,89100,9900,G,G-4,33.595248,130.633567,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,r2766,h_56,c_698,2017-04-08 16:41:45,2017-05-03,12:30:00,2017-05-06,1,95700,191400,287100,31900,C,C-3,38.435640,140.898354,True
442,r2983,h_56,c_753,2016-06-18 23:17:50,2016-07-13,10:00:00,2016-07-14,1,31900,63800,95700,31900,C,C-3,38.435640,140.898354,True
443,r2839,h_231,c_715,2016-04-06 02:51:01,2016-04-08,09:00:00,2016-04-10,1,29200,58400,87600,14600,A,A-3,35.914151,139.837520,True
444,r3476,h_43,c_866,2016-09-17 21:09:02,2016-10-03,09:30:00,2016-10-04,1,18100,36200,54300,18100,B,B-1,35.437093,139.799077,True


# Groupby・aggregation

## polars

In [39]:
%%time

result = (
    pl_reserve_tb
        .groupby(['hotel_id'])
        .agg([pl.count('reserve_id').alias('rsv_cnt'), 
              pl.n_unique('customer_id').alias('cus_cnt'),
              ])
        .sort('hotel_id')
    )

result

CPU times: user 3.46 ms, sys: 0 ns, total: 3.46 ms
Wall time: 9.61 ms


hotel_id,rsv_cnt,cus_cnt
str,u32,u32
"""h_1""",10,10
"""h_10""",3,3
"""h_100""",20,19
"""h_101""",17,17
"""h_102""",13,13
"""h_103""",10,10
"""h_104""",11,11
"""h_105""",15,15
"""h_106""",9,9


## pandas

In [40]:
%%time

result = (
    pd_reserve_tb
        .groupby('hotel_id')
        .agg({'reserve_id':'count', 'customer_id':'nunique'})
    )

result.reset_index(inplace=True)
result.columns = ['hotel_id','rsv_cnt','cus_cnt']
result

CPU times: user 7.65 ms, sys: 828 µs, total: 8.48 ms
Wall time: 9.23 ms


,hotel_id,rsv_cnt,cus_cnt
0,h_1,10,10
1,h_10,3,3
2,h_100,20,19
3,h_101,17,17
4,h_102,13,13
...,...,...,...
295,h_95,13,13
296,h_96,13,13
297,h_97,16,16
298,h_98,17,16


# 条件文で値を指定

## polars

In [41]:
%%time

def func(x):
    if  x < 200000:
        return '01_lowprice'
    elif x >= 200000 and x < 400000:
        return '02_midprice'
    elif x >= 400000:
        return '03_hiprice'
    else:
        return '99_missing'

tmp = pl_reserve_tb

tmp = tmp.with_columns(pl.col('total_price').apply(func).alias('price_category'))

tmp.groupby(['price_category']).count()

CPU times: user 2.31 ms, sys: 2.05 ms, total: 4.36 ms
Wall time: 6.33 ms


price_category,count
str,u32
"""01_lowprice""",3489
"""02_midprice""",420
"""03_hiprice""",121


In [42]:
%%time

# エクスプレションをつかった方がおすすめ
cond = (
    pl.when(pl.col("total_price") < 200000).then("01_lowprice")
    .when((pl.col("total_price") >= 200000)&(pl.col("total_price") < 400000) ).then("02_midprice")
    .when(pl.col("total_price") >= 400000).then("03_hiprice")
    .otherwise("99_missing"))

tmp = pl_reserve_tb

tmp = tmp.with_columns(cond.alias('price_category'))

tmp.groupby(['price_category']).count()

CPU times: user 2.81 ms, sys: 0 ns, total: 2.81 ms
Wall time: 3.03 ms


price_category,count
str,u32
"""01_lowprice""",3489
"""03_hiprice""",121
"""02_midprice""",420


## pandas

In [43]:
%%time

def func(x):
    if  x < 200000:
        return '01_lowprice'
    elif x >= 200000 and x < 400000:
        return '02_midprice'
    elif x >= 400000:
        return '03_hiprice'
    else:
        return '99_missing'

tmp = pd_reserve_tb

tmp['price_category'] = tmp['total_price'].apply(func)

tmp.groupby('price_category').agg({'reserve_id':'count'})

CPU times: user 6.42 ms, sys: 0 ns, total: 6.42 ms
Wall time: 6.54 ms


,reserve_id
price_category,
01_lowprice,3489
02_midprice,420
03_hiprice,121


# 値の置換

## polars

In [44]:
def replace(column: str, mapping: dict) -> pl.internals.expr.Expr:
    """
    Create a polars expression that replaces a columns values.

    Parameters
    ----------
    column : str
        Column name on which values should be replaced.
    mapping : dict
        Can be used to specify different replacement values for different existing values. For example,
        ``{'a': 'b', 'y': 'z'}`` replaces the value ‘a’ with ‘b’ and ‘y’ with ‘z’. Values not mentioned in ``mapping``
        will stay the same.

    Returns
    -------
    pl.internals.expr.Expr
        Expression that contains instructions to replace values in ``column`` according to ``mapping``.

    Raises
    ------
    Exception
        * If ``mapping`` is empty.
    TypeError
        * If ``column`` is not ``str``.
        * If ``mapping`` is not ``dict``.
    polars.exceptions.PanicException
        * When ``mapping`` has keys or values that are not mappable to arrows format. Only catchable via BaseException.
          See also https://pola-rs.github.io/polars-book/user-guide/datatypes.html.

    Examples
    --------
    >>> import polars as pl
    >>> df = pl.DataFrame({'fruit':['banana', 'apple', 'pie']})
    >>> df
    shape: (3, 1)
    ┌────────┐
    │ fruit  │
    │ ---    │
    │ str    │
    ╞════════╡
    │ banana │
    ├╌╌╌╌╌╌╌╌┤
    │ apple  │
    ├╌╌╌╌╌╌╌╌┤
    │ apple  │
    └────────┘
    >>> df.with_column(replace(column='fruit', mapping={'apple': 'pomegranate'}))
    shape: (3, 1)
    ┌─────────────┐
    │ fruit       │
    │ ---         │
    │ str         │
    ╞═════════════╡
    │ banana      │
    ├╌╌╌╌╌╌╌╌╌╌╌╌╌┤
    │ pomegranate │
    ├╌╌╌╌╌╌╌╌╌╌╌╌╌┤
    │ pomegranate │
    └─────────────┘

    """
    if not mapping:
        raise Exception("Mapping can't be empty")
    elif not isinstance(mapping, dict):
        TypeError(f"mapping must be of type dict, but is type: {type(mapping)}")
    if not isinstance(column, str):
        raise TypeError(f"column must be of type str, but is type: {type(column)}")

    branch = pl.when(pl.col(column) == list(mapping.keys())[0]).then(
        list(mapping.values())[0]
    )

    for from_value, to_value in mapping.items():
        branch = branch.when(pl.col(column) == from_value).then(to_value)

    return branch.otherwise(pl.col(column)).alias(column)

In [45]:
%%time

tmp_dict = {'man':'01_男性','woman':'02_女性'}

tmp = pl_customer_tb
tmp = tmp.with_columns(replace(column='sex', mapping=tmp_dict).alias('sex_rename'))

tmp.groupby('sex_rename').count()

CPU times: user 1.77 ms, sys: 0 ns, total: 1.77 ms
Wall time: 2.05 ms


sex_rename,count
str,u32
"""02_女性""",482
"""01_男性""",518


## pandas

In [46]:
%%time

tmp_dict = {'man':'01_男性','woman':'02_女性'}

tmp = pd_customer_tb
tmp['sex_rename'] = tmp['sex'].replace(tmp_dict)

tmp.groupby('sex_rename').agg({'customer_id':'count'})

CPU times: user 3.41 ms, sys: 956 µs, total: 4.36 ms
Wall time: 4.38 ms


,customer_id
sex_rename,
01_男性,518
02_女性,482


# 日時データの取り扱い

## polars

In [47]:
# 日時型、日付型への変換

# to_datetime関数で、datetime64[ns]型に変換
pl_reserve_tb.get_column('reserve_datetime').str.strptime(pl.Datetime, fmt='%Y-%m-%d %H:%M:%S')
(pl_reserve_tb['checkin_date'] +' '+ pl_reserve_tb['checkin_time']).str.strptime(pl.Datetime, fmt='%Y-%m-%d %H:%M:%S')

# datetime64[ns]型から日次情報を取得

print('reserve date： ')
print(pl_reserve_tb.get_column('reserve_datetime').str.strptime(pl.Datetime, fmt='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d'))
print('='*20)

print('checkin date： ')
print((pl_reserve_tb['checkin_date'] +' '+ pl_reserve_tb['checkin_time']).str.strptime(pl.Datetime, fmt='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d'))
print('='*20)

reserve date： 
shape: (4030,)
Series: 'reserve_datetime' [str]
[
	"2016-03-06"
	"2016-07-16"
	"2016-09-24"
	"2017-03-08"
	"2017-09-05"
	"2017-11-27"
	"2017-12-29"
	"2018-05-26"
	"2016-03-05"
	"2016-06-25"
	"2016-11-19"
	"2017-05-24"
	...
	"2017-08-20"
	"2017-12-10"
	"2018-04-22"
	"2016-01-08"
	"2016-05-13"
	"2016-04-21"
	"2016-10-06"
	"2017-03-11"
	"2017-06-27"
	"2017-09-29"
	"2018-03-14"
	"2016-04-16"
	"2016-06-06"
]
checkin date： 
shape: (4030,)
Series: 'checkin_date' [str]
[
	"2016-03-26"
	"2016-07-20"
	"2016-10-19"
	"2017-03-29"
	"2017-09-22"
	"2017-12-04"
	"2018-01-25"
	"2018-06-08"
	"2016-03-25"
	"2016-07-14"
	"2016-12-08"
	"2017-06-20"
	...
	"2017-09-06"
	"2017-12-20"
	"2018-05-21"
	"2016-01-09"
	"2016-06-06"
	"2016-04-22"
	"2016-10-07"
	"2017-03-27"
	"2017-07-10"
	"2017-10-09"
	"2018-04-02"
	"2016-05-10"
	"2016-07-06"
]


In [48]:
# 年/月/日/時刻/分/秒/曜日への変換

tmp = pl_reserve_tb

# reserve_datetimeをdatetime64[ns]型に変換
tmp = tmp.with_columns(pl.col('reserve_datetime').str.strptime(pl.Datetime, fmt='%Y-%m-%d %H:%M:%S'))

# 年を取得
print('年： ')
# print(tmp.get_column('reserve_datetime').dt.year())
print(tmp.with_columns(pl.col('reserve_datetime').dt.year().alias('y')).select('y'))
print('='*20)

# 月を取得
print('月： ')
print(tmp.get_column('reserve_datetime').dt.month())
print('='*20)

# 日を取得
print('日： ')
print(tmp.get_column('reserve_datetime').dt.day())
print('='*20)

# # 曜日（0=日曜日、1＝月曜日）を数値で取得
# print('曜日： ')
# tmp_dict = {0:'月曜',1:'火曜',2:'水曜',3:'木曜',4:'金曜',5:'土曜',6:'日曜'}
# print(tmp['reserve_datetime'].dt.dayofweek.replace(tmp_dict))
# print('='*20)

# 時刻の時を取得
print('時刻： ')
print(tmp.get_column('reserve_datetime').dt.hour())
print('='*20)

# 時刻の分を取得
print('時刻の分： ')
print(tmp.get_column('reserve_datetime').dt.minute())
print('='*20)

# 時刻の秒を取得
print('時刻の秒： ')
print(tmp.get_column('reserve_datetime').dt.second())
print('='*20)

# 指定したフォーマットの文字列に変換
print('指定のフォーマットに文字変換後： ')
print(tmp.get_column('reserve_datetime').dt.strftime('%Y/%-m/%-d'))
print('='*20)

年： 
shape: (4030, 1)
┌──────┐
│ y    │
│ ---  │
│ i32  │
╞══════╡
│ 2016 │
│ 2016 │
│ 2016 │
│ 2017 │
│ ...  │
│ 2017 │
│ 2018 │
│ 2016 │
│ 2016 │
└──────┘
月： 
shape: (4030,)
Series: 'reserve_datetime' [u32]
[
	3
	7
	9
	3
	9
	11
	12
	5
	3
	6
	11
	5
	...
	8
	12
	4
	1
	5
	4
	10
	3
	6
	9
	3
	4
	6
]
日： 
shape: (4030,)
Series: 'reserve_datetime' [u32]
[
	6
	16
	24
	8
	5
	27
	29
	26
	5
	25
	19
	24
	...
	20
	10
	22
	8
	13
	21
	6
	11
	27
	29
	14
	16
	6
]
時刻： 
shape: (4030,)
Series: 'reserve_datetime' [u32]
[
	13
	23
	10
	3
	19
	18
	10
	8
	13
	9
	12
	10
	...
	17
	19
	20
	20
	9
	21
	18
	11
	23
	5
	5
	15
	8
]
時刻の分： 
shape: (4030,)
Series: 'reserve_datetime' [u32]
[
	9
	39
	3
	20
	50
	47
	38
	42
	31
	12
	49
	6
	...
	56
	39
	41
	30
	8
	50
	1
	56
	0
	24
	1
	20
	16
]
時刻の秒： 
shape: (4030,)
Series: 'reserve_datetime' [u32]
[
	42
	55
	17
	10
	37
	5
	36
	51
	6
	22
	10
	21
	...
	37
	53
	29
	10
	55
	14
	34
	5
	2
	57
	45
	17
	51
]
指定のフォーマットに文字変換後： 
shape: (4030,)
Series: 'reserve_datetime' [str]
[
	"2016/3/

In [49]:
# 日時差を取得

tmp = pl_reserve_tb

# reserve_datetimeをdatetime64[ns]型に変換
tmp = tmp.with_columns(pl.col('reserve_datetime').str.strptime(pl.Datetime, fmt='%Y-%m-%d %H:%M:%S'))

# checkin_datetimeをdatetime64[ns]型に変換
tmp = tmp.with_columns((pl_reserve_tb['checkin_date'] +' '+ pl_reserve_tb['checkin_time']).str.strptime(pl.Datetime, fmt='%Y-%m-%d %H:%M:%S').alias('checkin_datetime'))

# 年の差分を計算（月以下の日時要素は考慮しない）
print('年の差分を計算（月以下の日時要素は考慮しない）： ')
print(tmp['reserve_datetime'].dt.year() - tmp['checkin_datetime'].dt.year())
print('='*20)

# # 月の差分を取得（日以下の日時要素は考慮しない）
print('月の差分を取得（日以下の日時要素は考慮しない）： ')
print((tmp['reserve_datetime'].dt.year() * 12 +tmp['reserve_datetime'].dt.month()) - (tmp['checkin_datetime'].dt.year() * 12 +tmp['checkin_datetime'].dt.month()))
print('='*20)

"""
<工事中>

# 日単位で差分を計算
print('日単位で差分を計算： ')
print((tmp['reserve_datetime'] - tmp['checkin_datetime']))
print('='*20)

# # 時単位で差分を計算
# print('時単位で差分を計算： ')
# print((tmp['reserve_datetime'] - tmp['checkin_datetime']).astype('timedelta64[h]'))
# print('='*20)

# # 分単位で差分を計算
# print('分単位で差分を計算： ')
# print((tmp['reserve_datetime'] - tmp['checkin_datetime']).astype('timedelta64[m]'))
# print('='*20)

# # 秒単位で差分を計算
# print('秒単位で差分を計算： ')
# print((tmp['reserve_datetime'] - tmp['checkin_datetime']).astype('timedelta64[s]'))
# print('='*20)

"""

年の差分を計算（月以下の日時要素は考慮しない）： 
shape: (4030,)
Series: 'reserve_datetime' [i32]
[
	0
	0
	0
	0
	0
	0
	-1
	0
	0
	0
	0
	0
	...
	0
	0
	0
	0
	0
	0
	0
	0
	0
	0
	0
	0
	0
]
月の差分を取得（日以下の日時要素は考慮しない）： 
shape: (4030,)
Series: 'reserve_datetime' [i64]
[
	0
	0
	-1
	0
	0
	-1
	-1
	-1
	0
	-1
	-1
	-1
	...
	-1
	0
	-1
	0
	-1
	0
	0
	0
	-1
	-1
	-1
	-1
	-1
]


"\n<工事中>\n\n# 日単位で差分を計算\nprint('日単位で差分を計算： ')\nprint((tmp['reserve_datetime'] - tmp['checkin_datetime']))\nprint('='*20)\n\n# # 時単位で差分を計算\n# print('時単位で差分を計算： ')\n# print((tmp['reserve_datetime'] - tmp['checkin_datetime']).astype('timedelta64[h]'))\n# print('='*20)\n\n# # 分単位で差分を計算\n# print('分単位で差分を計算： ')\n# print((tmp['reserve_datetime'] - tmp['checkin_datetime']).astype('timedelta64[m]'))\n# print('='*20)\n\n# # 秒単位で差分を計算\n# print('秒単位で差分を計算： ')\n# print((tmp['reserve_datetime'] - tmp['checkin_datetime']).astype('timedelta64[s]'))\n# print('='*20)\n\n"

In [50]:
"""
<工事中>

import datetime
from dateutil.relativedelta import relativedelta

# 日時型の増減

tmp = pd_reserve_tb

# reserve_datetimeをdatetime64[ns]型に変換
tmp['reserve_datetime'] = pd.to_datetime(tmp['reserve_datetime'], format='%Y-%m-%d %H:%M:%S')

# 日時を加算(+1m20day)
(tmp['reserve_datetime'].map(lambda x: x + relativedelta(months=1)) + datetime.timedelta(days=20)).dt.date"""

"\n<工事中>\n\nimport datetime\nfrom dateutil.relativedelta import relativedelta\n\n# 日時型の増減\n\ntmp = pd_reserve_tb\n\n# reserve_datetimeをdatetime64[ns]型に変換\ntmp['reserve_datetime'] = pd.to_datetime(tmp['reserve_datetime'], format='%Y-%m-%d %H:%M:%S')\n\n# 日時を加算(+1m20day)\n(tmp['reserve_datetime'].map(lambda x: x + relativedelta(months=1)) + datetime.timedelta(days=20)).dt.date"

## pandas



*   pandasにはdatetime64[ns]型とdatetime64[D]型がある（[xx]のxxは扱う最小単位を表す）
*   日時要素を取り出すときはdatetime64[ns]型に変換後に日次要素を取得する方が便利

In [51]:
# 日時型、日付型への変換

# to_datetime関数で、datetime64[ns]型に変換
pd.to_datetime(pd_reserve_tb['reserve_datetime'], format='%Y-%m-%d %H:%M:%S')
pd.to_datetime(pd_reserve_tb['checkin_date'] +' '+ pd_reserve_tb['checkin_time'], format='%Y-%m-%d %H:%M:%S')

# datetime64[ns]型から日次情報を取得

print('reserve date： ')
print(pd.to_datetime(pd_reserve_tb['reserve_datetime'],
               format='%Y-%m-%d %H:%M:%S').dt.date)
print('='*20)

print('checkin date： ')
print(pd.to_datetime(pd_reserve_tb['checkin_date'] +' '+ pd_reserve_tb['checkin_time'], format='%Y-%m-%d %H:%M:%S').dt.date)
print('='*20)

reserve date： 
0       2016-03-06
1       2016-07-16
2       2016-09-24
3       2017-03-08
4       2017-09-05
           ...    
4025    2017-06-27
4026    2017-09-29
4027    2018-03-14
4028    2016-04-16
4029    2016-06-06
Name: reserve_datetime, Length: 4030, dtype: object
checkin date： 
0       2016-03-26
1       2016-07-20
2       2016-10-19
3       2017-03-29
4       2017-09-22
           ...    
4025    2017-07-10
4026    2017-10-09
4027    2018-04-02
4028    2016-05-10
4029    2016-07-06
Length: 4030, dtype: object




*   上記のフォーマット文字で二桁表示のところは全て律儀に、左側ゼロ埋めで二桁で表示されます。例えば 4月を表す場合に %m とすると 04 となります。
*    もし 04 ではなく 4 としたい場合は、%-m のようにハイフンを付けます。
*    https://python.keicode.com/lang/format-datetime.php



In [52]:
# 年/月/日/時刻/分/秒/曜日への変換

tmp = pd_reserve_tb

# reserve_datetimeをdatetime64[ns]型に変換
tmp['reserve_datetime'] = pd.to_datetime(pd_reserve_tb['reserve_datetime'], format='%Y-%m-%d %H:%M:%S')

# 年を取得
print('年： ')
print(tmp['reserve_datetime'].dt.year)
print('='*20)

# 月を取得
print('月： ')
print(tmp['reserve_datetime'].dt.month)
print('='*20)

# 日を取得
print('日： ')
print(tmp['reserve_datetime'].dt.day)
print('='*20)

# 曜日（0=日曜日、1＝月曜日）を数値で取得
print('曜日： ')
tmp_dict = {0:'月曜',1:'火曜',2:'水曜',3:'木曜',4:'金曜',5:'土曜',6:'日曜'}
print(tmp['reserve_datetime'].dt.dayofweek.replace(tmp_dict))
print('='*20)

# 時刻の時を取得
print('時刻： ')
print(tmp['reserve_datetime'].dt.hour)
print('='*20)

# 時刻の分を取得
print('時刻の分： ')
print(tmp['reserve_datetime'].dt.minute)
print('='*20)

# 時刻の秒を取得
print('時刻の秒： ')
print(tmp['reserve_datetime'].dt.second)
print('='*20)

# 指定したフォーマットの文字列に変換
print('指定のフォーマットに文字変換後： ')
print(tmp['reserve_datetime'].dt.strftime('%Y/%-m/%-d'))
print('='*20)

年： 
0       2016
1       2016
2       2016
3       2017
4       2017
        ... 
4025    2017
4026    2017
4027    2018
4028    2016
4029    2016
Name: reserve_datetime, Length: 4030, dtype: int64
月： 
0       3
1       7
2       9
3       3
4       9
       ..
4025    6
4026    9
4027    3
4028    4
4029    6
Name: reserve_datetime, Length: 4030, dtype: int64
日： 
0        6
1       16
2       24
3        8
4        5
        ..
4025    27
4026    29
4027    14
4028    16
4029     6
Name: reserve_datetime, Length: 4030, dtype: int64
曜日： 
0       日曜
1       土曜
2       土曜
3       水曜
4       火曜
        ..
4025    火曜
4026    金曜
4027    水曜
4028    土曜
4029    月曜
Name: reserve_datetime, Length: 4030, dtype: object
時刻： 
0       13
1       23
2       10
3        3
4       19
        ..
4025    23
4026     5
4027     5
4028    15
4029     8
Name: reserve_datetime, Length: 4030, dtype: int64
時刻の分： 
0        9
1       39
2        3
3       20
4       50
        ..
4025     0
4026    24
4027     1


In [53]:
# 日時差を取得

tmp = pd_reserve_tb

# reserve_datetimeをdatetime64[ns]型に変換
tmp['reserve_datetime'] = pd.to_datetime(tmp['reserve_datetime'], format='%Y-%m-%d %H:%M:%S')

# checkin_datetimeをdatetime64[ns]型に変換
tmp['checkin_datetime'] = pd.to_datetime(tmp['checkin_date'] +' '+ tmp['checkin_time'], format='%Y-%m-%d %H:%M:%S')

# 年の差分を計算（月以下の日時要素は考慮しない）
print('年の差分を計算（月以下の日時要素は考慮しない）： ')
print(tmp['reserve_datetime'].dt.year - tmp['checkin_datetime'].dt.year)
print('='*20)

# 月の差分を取得（日以下の日時要素は考慮しない）
print('月の差分を取得（日以下の日時要素は考慮しない）： ')
print((tmp['reserve_datetime'].dt.year * 12 +tmp['reserve_datetime'].dt.month) - (tmp['checkin_datetime'].dt.year * 12 +tmp['checkin_datetime'].dt.month))
print('='*20)

# 日単位で差分を計算
print('日単位で差分を計算： ')
print((tmp['reserve_datetime'] - tmp['checkin_datetime']).astype('timedelta64[D]'))
print('='*20)

# 時単位で差分を計算
print('時単位で差分を計算： ')
print((tmp['reserve_datetime'] - tmp['checkin_datetime']).astype('timedelta64[h]'))
print('='*20)

# 分単位で差分を計算
print('分単位で差分を計算： ')
print((tmp['reserve_datetime'] - tmp['checkin_datetime']).astype('timedelta64[m]'))
print('='*20)

# 秒単位で差分を計算
print('秒単位で差分を計算： ')
print((tmp['reserve_datetime'] - tmp['checkin_datetime']).astype('timedelta64[s]'))
print('='*20)

年の差分を計算（月以下の日時要素は考慮しない）： 
0       0
1       0
2       0
3       0
4       0
       ..
4025    0
4026    0
4027    0
4028    0
4029    0
Length: 4030, dtype: int64
月の差分を取得（日以下の日時要素は考慮しない）： 
0       0
1       0
2      -1
3       0
4       0
       ..
4025   -1
4026   -1
4027   -1
4028   -1
4029   -1
Length: 4030, dtype: int64
日単位で差分を計算： 
0      -20.0
1       -4.0
2      -25.0
3      -22.0
4      -17.0
        ... 
4025   -13.0
4026   -11.0
4027   -20.0
4028   -24.0
4029   -31.0
Length: 4030, dtype: float64
時単位で差分を計算： 
0      -477.0
1       -84.0
2      -599.0
3      -512.0
4      -399.0
        ...  
4025   -299.0
4026   -246.0
4027   -463.0
4028   -571.0
4029   -721.0
Length: 4030, dtype: float64
分単位で差分を計算： 
0      -28611.0
1       -5031.0
2      -35937.0
3      -30700.0
4      -23920.0
         ...   
4025   -17910.0
4026   -14706.0
4027   -27749.0
4028   -34210.0
4029   -43244.0
Length: 4030, dtype: float64
秒単位で差分を計算： 
0      -1716618.0
1       -301805.0
2      -2156203.0
3      -1841

In [54]:
import datetime
from dateutil.relativedelta import relativedelta

# 日時型の増減

tmp = pd_reserve_tb

# reserve_datetimeをdatetime64[ns]型に変換
tmp['reserve_datetime'] = pd.to_datetime(tmp['reserve_datetime'], format='%Y-%m-%d %H:%M:%S')

# 日時を加算(+1m20day)
(tmp['reserve_datetime'].map(lambda x: x + relativedelta(months=1)) + datetime.timedelta(days=20)).dt.date

0       2016-04-26
1       2016-09-05
2       2016-11-13
3       2017-04-28
4       2017-10-25
           ...    
4025    2017-08-16
4026    2017-11-18
4027    2018-05-04
4028    2016-06-05
4029    2016-07-26
Name: reserve_datetime, Length: 4030, dtype: object

# pivot

## polars

*    DataFrame.pivot()でピボットテーブルを作成します。引数はvalue=・index=・columns=・aggregate_fn=の順に渡します。pandasのDataFrame.pivot_table()に相当します。
*    なお、ピボットテーブルを戻す場合、（pandasでいう）.stack()は存在しませんが、.melt()が存在します。

In [57]:
# <工事中>

# 顧客ID/宿泊人数毎に予約数をカウント
# pl_reserve_tb.pivot(index='customer_id', columns='people_num', 
#                     values='reserve_id',
#                     aggregate_fn=lambda x: len(x))

AttributeError: ignored

## pandas

In [58]:
# 顧客ID/宿泊人数毎に予約数をカウント
pd.pivot_table(pd_reserve_tb, index='customer_id', columns='people_num', 
               values='reserve_id',
               aggfunc=lambda x: len(x), fill_value=0)

people_num,1,2,3,4
customer_id,,,,
c_1,2,2,2,2
c_10,0,2,2,2
c_100,2,1,2,0
c_1000,1,0,0,1
c_101,2,1,1,1
...,...,...,...,...
c_994,1,0,0,0
c_995,2,2,1,3
c_996,0,4,3,0


# ランダムサンプリング

## polars

In [59]:
pl_reserve_tb.sample(100)

reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
str,str,str,str,str,str,str,i64,i64
"""r1833""","""h_240""","""c_453""","""2017-06-07 21:...","""2017-06-26""","""12:30:00""","""2017-06-28""",4,213600
"""r2407""","""h_262""","""c_603""","""2016-01-05 16:...","""2016-01-25""","""10:00:00""","""2016-01-28""",2,114000
"""r1382""","""h_23""","""c_338""","""2016-03-24 20:...","""2016-04-01""","""12:30:00""","""2016-04-02""",2,130200
"""r1502""","""h_43""","""c_369""","""2017-04-30 01:...","""2017-05-20""","""09:30:00""","""2017-05-22""",2,72400
"""r3209""","""h_42""","""c_806""","""2017-01-31 11:...","""2017-02-28""","""09:30:00""","""2017-03-01""",3,33900
"""r257""","""h_20""","""c_60""","""2016-01-04 06:...","""2016-01-15""","""09:00:00""","""2016-01-17""",4,56800
"""r1587""","""h_117""","""c_388""","""2017-04-05 13:...","""2017-04-21""","""09:00:00""","""2017-04-22""",2,29400
"""r1214""","""h_37""","""c_296""","""2016-08-31 12:...","""2016-09-08""","""12:00:00""","""2016-09-10""",3,150600
"""r2092""","""h_229""","""c_523""","""2016-11-03 20:...","""2016-11-19""","""11:00:00""","""2016-11-20""",3,22200


## pandas

In [60]:
pd_reserve_tb.sample(100)

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,total_price_2x,total_price_3x,price_category,checkin_datetime
530,r531,h_81,c_124,2017-10-09 09:59:56,2017-10-19,09:30:00,2017-10-21,2,49600,99200,148800,01_lowprice,2017-10-19 09:30:00
3619,r3620,h_177,c_903,2016-06-26 22:38:23,2016-06-30,12:00:00,2016-07-02,4,560800,1121600,1682400,03_hiprice,2016-06-30 12:00:00
2963,r2964,h_292,c_748,2017-04-10 11:49:28,2017-04-13,11:00:00,2017-04-14,4,36400,72800,109200,01_lowprice,2017-04-13 11:00:00
2882,r2883,h_70,c_725,2016-12-04 13:13:56,2016-12-21,11:00:00,2016-12-22,1,7800,15600,23400,01_lowprice,2016-12-21 11:00:00
603,r604,h_169,c_146,2017-10-27 16:23:11,2017-11-15,10:30:00,2017-11-18,4,100800,201600,302400,01_lowprice,2017-11-15 10:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3689,r3690,h_223,c_918,2017-07-14 08:42:53,2017-08-01,11:00:00,2017-08-03,4,548000,1096000,1644000,03_hiprice,2017-08-01 11:00:00
1818,r1819,h_236,c_450,2016-01-02 18:03:54,2016-01-31,10:30:00,2016-02-03,2,93000,186000,279000,01_lowprice,2016-01-31 10:30:00
2447,r2448,h_227,c_611,2017-04-02 00:03:21,2017-04-22,11:00:00,2017-04-24,4,63200,126400,189600,01_lowprice,2017-04-22 11:00:00
2195,r2196,h_188,c_548,2016-08-13 13:35:46,2016-09-06,12:30:00,2016-09-08,3,355200,710400,1065600,02_midprice,2016-09-06 12:30:00
